In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import warnings; warnings.simplefilter('ignore')

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [10]:
df_review_r = pd.read_csv('Yelp_Dataset_Reviews_reduced.csv',index_col=None)
df_review_r.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,review_id,user_id,business_id,rating,useful,funny,cool,text,date,year
0,0,0,0.0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6.0,1.0,0.0,total bill for this horrible service over 8gs ...,2013-05-07 04:34:36,2013.0
1,4,4,4.0,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,7.0,0.0,0.0,today was my second out of three sessions i ha...,2018-01-30 23:07:38,2018.0
2,6,6,6.0,G7XHMxG0bx9oBJNECG4IFg,jlu4CztcSxrKx56ba1a5AQ,3fw2X5bZYeW9xCz_zGhOHg,3.0,5.0,4.0,5.0,tracy dessert had a big name in hong kong and ...,2016-05-07 01:21:02,2016.0
3,17,17,17.0,6BnQwlxRn7ZuWdzninM9sQ,JSrP-dUmLlwZiI7Dp3PQ2A,cHdJXLlKNWixBXpDwEGb_A,3.0,1.0,7.0,1.0,i love chinese food and i love mexican food wh...,2015-04-01 16:30:00,2015.0
4,21,21,21.0,IPw8yWiyqnfBzzWmypUHgg,5JVY32_bmTBfIGpCCsnAfw,Mem13A3C202RzT53npn4NA,5.0,9.0,6.0,9.0,if you are looking for the best pierogies in p...,2017-05-13 10:41:43,2017.0


In [24]:
# df_review_r = df_review_r.drop(['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1'],axis=1)
print(len(df_review_r))
df_review_r = df_review_r.rename(columns={'rating':'review rating'})
df_review_r.head()

498755


,review_id,user_id,business_id,review rating,useful,funny,cool,text,date,year
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6.0,1.0,0.0,total bill for this horrible service over 8gs ...,2013-05-07 04:34:36,2013.0
1,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,7.0,0.0,0.0,today was my second out of three sessions i ha...,2018-01-30 23:07:38,2018.0
2,G7XHMxG0bx9oBJNECG4IFg,jlu4CztcSxrKx56ba1a5AQ,3fw2X5bZYeW9xCz_zGhOHg,3.0,5.0,4.0,5.0,tracy dessert had a big name in hong kong and ...,2016-05-07 01:21:02,2016.0
3,6BnQwlxRn7ZuWdzninM9sQ,JSrP-dUmLlwZiI7Dp3PQ2A,cHdJXLlKNWixBXpDwEGb_A,3.0,1.0,7.0,1.0,i love chinese food and i love mexican food wh...,2015-04-01 16:30:00,2015.0
4,IPw8yWiyqnfBzzWmypUHgg,5JVY32_bmTBfIGpCCsnAfw,Mem13A3C202RzT53npn4NA,5.0,9.0,6.0,9.0,if you are looking for the best pierogies in p...,2017-05-13 10:41:43,2017.0


In [26]:
df_review_r = df_review_r.drop(['useful','funny','cool','date','year'], axis=1)
df_review_r.head()

,review_id,user_id,business_id,review rating,text
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,total bill for this horrible service over 8gs ...
1,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,today was my second out of three sessions i ha...
2,G7XHMxG0bx9oBJNECG4IFg,jlu4CztcSxrKx56ba1a5AQ,3fw2X5bZYeW9xCz_zGhOHg,3.0,tracy dessert had a big name in hong kong and ...
3,6BnQwlxRn7ZuWdzninM9sQ,JSrP-dUmLlwZiI7Dp3PQ2A,cHdJXLlKNWixBXpDwEGb_A,3.0,i love chinese food and i love mexican food wh...
4,IPw8yWiyqnfBzzWmypUHgg,5JVY32_bmTBfIGpCCsnAfw,Mem13A3C202RzT53npn4NA,5.0,if you are looking for the best pierogies in p...


In [22]:
df_biz = pd.read_csv('Yelp_Dataset_Business_Info_new.csv')
df_biz = df_biz.drop('Unnamed: 0', axis=1)
df_biz = df_biz[['business_id','name','rating','review_count','categories']]
# df_biz['business rating'] = df_biz['rating']
df_biz = df_biz.rename(columns={'rating': 'business rating'})
print(len(df_biz))
df_biz.head()

158525


,business_id,name,business rating,review_count,categories
0,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,2.5,128,"Specialty Food, Restaurants, Dim Sum, Imported..."
1,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,4.0,170,"Sushi Bars, Restaurants, Japanese"
2,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,5.0,3,"Insurance, Financial Services"
3,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,4.0,4,"Plumbing, Shopping, Local Services, Home Servi..."
4,68dUKd8_8liJ7in4aWOSEA,The UPS Store,2.5,3,"Shipping Centers, Couriers & Delivery Services..."


In [35]:
print('Number of unique businesses: ',len(df_review_r.business_id.unique()))
print('Number of unique review users: ',len(df_review_r.user_id.unique()))
print('Number of unique reviews: ',len(df_review_r.review_id.unique()))

Number of unique businesses:  96645
Number of unique review users:  177669
Number of unique reviews:  498749


In [34]:
biz_dict = df_biz.to_dict(orient='records')
biz_dict[0]

{'business_id': 'QXAEGFB4oINsVuTFxEYKFQ',
 'name': 'Emerald Chinese Restaurant',
 'business rating': 2.5,
 'review_count': 128,
 'categories': 'Specialty Food, Restaurants, Dim Sum, Imported Food, Food, Chinese, Ethnic Food, Seafood'}